<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-7.-Function-Decorators-and-Closures" data-toc-modified-id="Chapter-7.-Function-Decorators-and-Closures-1">Chapter 7. Function Decorators and Closures</a></span><ul class="toc-item"><li><span><a href="#Decorator-Enhanced-Strategy-Pattern" data-toc-modified-id="Decorator-Enhanced-Strategy-Pattern-1.1">Decorator-Enhanced Strategy Pattern</a></span></li><li><span><a href="#Local-Variables-in-functions" data-toc-modified-id="Local-Variables-in-functions-1.2">Local Variables in functions</a></span></li><li><span><a href="#Closures" data-toc-modified-id="Closures-1.3">Closures</a></span></li><li><span><a href="#non-local-variables" data-toc-modified-id="non-local-variables-1.4">non-local variables</a></span><ul class="toc-item"><li><span><a href="#keyword-argument-support-with-@functools.wraps" data-toc-modified-id="keyword-argument-support-with-@functools.wraps-1.4.1">keyword argument support with @functools.wraps</a></span></li><li><span><a href="#Memoization-with-functools.lru_cache()" data-toc-modified-id="Memoization-with-functools.lru_cache()-1.4.2">Memoization with functools.lru_cache()</a></span></li><li><span><a href="#Generic-Functions-using-functools.singledispatch()" data-toc-modified-id="Generic-Functions-using-functools.singledispatch()-1.4.3">Generic Functions using functools.singledispatch()</a></span></li></ul></li><li><span><a href="#Stacked-Decorators" data-toc-modified-id="Stacked-Decorators-1.5">Stacked Decorators</a></span></li><li><span><a href="#Parameterized-Decorators" data-toc-modified-id="Parameterized-Decorators-1.6">Parameterized Decorators</a></span><ul class="toc-item"><li><span><a href="#The-Parameterized-Clock-Decorator" data-toc-modified-id="The-Parameterized-Clock-Decorator-1.6.1">The Parameterized Clock Decorator</a></span></li></ul></li></ul></li></ul></div>

# Chapter 7. Function Decorators and Closures
A decorator is a (higher order) callable that takes another function as argument (the decorated function). The decorator may perform some processing with the decorated function, and returns it or replaces it with another function or callable object.


`@decorate
def target():
    print('running target()')`

is the same as

`def target():
    print('running target()')`

`target = decorate(target)`

In [223]:
def deco(func):
    def inner():
        print('running inner()')
        func()
    return inner  

@deco
def target():  
    print('running target()')

In [224]:
target()

running inner()
running target()


decorators run right after the decorated function is defined usually at import time. The decorated function is passed as a parameter to the decorator

In [168]:
registry = []   

def register(func):   
    print('running register(%s)' % func)   
    registry.append(func)   
    return func   # In practice, most decorators 
                  # define an inner function and return it.

@register   
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():   
    print('running f3()')

def main():   
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__=='__main__':
    main()   

running register(<function f1 at 0x7f8a0365dc10>)
running register(<function f2 at 0x7f8a0365ddc0>)
running main()
registry -> [<function f1 at 0x7f8a0365dc10>, <function f2 at 0x7f8a0365ddc0>]
running f1()
running f2()
running f3()


## Decorator-Enhanced Strategy Pattern

In [174]:
# Any function decorated by @promotion will be added to promos

promos = []
def promotion(promo_func):   
    promos.append(promo_func)
    return promo_func

@promotion   
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):   
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

## Local Variables in functions

In [175]:
# Variable b is local, because it is assigned a 
# value in the body of the function

b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [176]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

If we want the interpreter to treat b as a global variable in spite of the assignment within the function, we use the global declaration

In [177]:
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [178]:
f3(3)

3
6


## Closures
a closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there.

In [179]:
#  A callable object to calculate a running average
class Averager():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [181]:
avg = Averager()
[avg(i) for i in range(1,10)]

[1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

In [182]:
#  A closure to calculate a running average
def make_averager():
    series = [] # series is not bound in the local scope (free)
                # so exists after the local scope is gone

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

In [184]:
avg = make_averager()
[avg(i) for i in range(1,10)]

[1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

series is not bound in the local scope (free variable) so exists after the local scope is gone

In [188]:
avg.__code__.co_varnames

('new_value', 'total')

In [189]:
avg.__code__.co_freevars

('series',)

In [190]:
avg.__closure__[0].cell_contents

[1, 2, 3, 4, 5, 6, 7, 8, 9]

## non-local variables
non-local keyword flags a variable as a free variable even when it is assigned a new value within the function. 

In [191]:
def make_averager():
    count = 0 # total and count are immutable types so are bound local variables
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

In [192]:

avg = make_averager()
[avg(i) for i in range(10)]

UnboundLocalError: local variable 'count' referenced before assignment

In [193]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [195]:
avg = make_averager()
[avg(i) for i in range(1,10)]

[1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

In [198]:
# A simple decorator to output the running time of functions
import time

def clock(func):
    def clocked(*args):  
        t0 = time.perf_counter()
        result = func(*args)  
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked  
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12357510s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000052s] factorial(1) -> 1
[0.00002578s] factorial(2) -> 2
[0.00004660s] factorial(3) -> 6
[0.00008424s] factorial(4) -> 24
[0.00011994s] factorial(5) -> 120
[0.00019614s] factorial(6) -> 720
6! = 720


### keyword argument support with @functools.wraps

In [ ]:
import time
import functools
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r ' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [ ]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

### Memoization with functools.lru_cache()
memoization: an optimization technique that works by saving the results of previous invocations of an expensive function.

The letters LRU stand for Least Recently Used, meaning that the growth of the cache is limited by discarding the entries that have not been read for a while (determined by the maxsize parameter)

generating the nth number in the Fibonacci sequence makes a lot of redundent calls without memoization

In [ ]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))


In [ ]:
@functools.lru_cache() 
@clock  # 
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

### Generic Functions using functools.singledispatch()

We want to be able to generate HTML displays for different types of Python objects without a chain of if/elif/elif calling specialized functions like htmlize_str, htmlize_int, etc

In [235]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch   
def htmlize(obj):
    print('calling generic version')
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)   
def _(text): 
    print('calling str version')
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)   
def _(n):
    print('calling Integral version')
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)   
@htmlize.register(abc.MutableSequence)
def _(seq):
    print('calling MutableSequence version')
    inner = '</li><li>'.join(htmlize(item) for item in seq)
    return '<ul><li>' + inner + '</li></ul>'

In [228]:
htmlize({1, 2, 3})

calling generic version


'<pre>{1, 2, 3}</pre>'

In [229]:
htmlize(abs)

calling generic version


'<pre>&lt;built-in function abs&gt;</pre>'

In [230]:
htmlize('Heimlich & Co.\n- a game')

calling str version


'<p>Heimlich &amp; Co.<br>\n- a game</p>'

In [231]:
htmlize(42)

calling Integral version


'<pre>42 (0x2a)</pre>'

In [236]:
htmlize(['alpha', 66, {3, 2, 1}])

calling MutableSequence version
calling str version
calling Integral version
calling generic version


'<ul><li><p>alpha</p></li><li><pre>66 (0x42)</pre></li><li><pre>{1, 2, 3}</pre></li></ul>'

<html>
<ul><ul><li><p>alpha</p></li><li><pre>66 (0x42)</pre></li><li><pre>{1, 2, 3}</pre></li></ul>
</html>

## Stacked Decorators

`@d1
@d2
def f():
    print('f')`
    
Is the same as:

`def f():
   print('f')`

`f = d1(d2(f))`

## Parameterized Decorators

In [237]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry ->', registry)
f1()

running register(<function f1 at 0x7f8a0365d8b0>)
running main()
registry -> [<function f1 at 0x7f8a0365d8b0>]
running f1()


In [238]:
# To accept parameters, the new register decorator 
# must be called as a function

registry = set()   

def register(active=True):   
    def decorate(func):   
        print('running register(active=%s)->decorate(%s)'
              % (active, func))
        if active:    
            registry.add(func)
        else:
            registry.discard(func)   

        return func   
    return decorate   

@register(active=False)   
def f1():
    print('running f1()')

@register()   
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    

running register(active=False)->decorate(<function f1 at 0x7f8a03cb9160>)
running register(active=True)->decorate(<function f2 at 0x7f8a038b6310>)


In [239]:
registry

{<function __main__.f2()>}

In [240]:
# call decorator directly passing parameter
register()(f3)

running register(active=True)->decorate(<function f3 at 0x7f8a0365d8b0>)


<function __main__.f3()>

In [241]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [242]:
# remove f2 from the registry
register(active=False)(f2)

running register(active=False)->decorate(<function f2 at 0x7f8a038b6310>)


<function __main__.f2()>

In [243]:
registry

{<function __main__.f3()>}

### The Parameterized Clock Decorator
users may pass a format string to control the output of the decorated function. 

In [244]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):   
    def decorate(func):       
        def clocked(*_args):  
            t0 = time.time()
            _result = func(*_args)   
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)   
            result = repr(_result)   
            print(fmt.format(**locals()))   
            return _result   
        return clocked   
    return decorate   

if __name__ == '__main__':

    @clock()   
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

[0.12757087s] snooze(0.123) -> None
[0.12673998s] snooze(0.123) -> None
[0.12771201s] snooze(0.123) -> None


In [245]:
import time

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12699294090270996s
snooze: 0.12494993209838867s
snooze: 0.12491393089294434s


In [246]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.124s
snooze(0.123) dt=0.125s
snooze(0.123) dt=0.128s
